In [18]:
multi_sentence_text = """The Monte Carlo algorithm (or Monte Carlo method) refers to a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. These methods are used in various fields such as finance, physics, engineering, and AI for problems that are deterministic in principle but difficult to solve directly.

Key Aspects of the Monte Carlo Algorithm:
Random Sampling: The core idea is to generate random variables to simulate complex systems or processes. The algorithm uses probabilistic models and generates numerous random samples to approximate the solution to a deterministic problem.

Estimation of Results: Monte Carlo methods provide estimates of the outcome by running many simulations and averaging the results. These results tend to get more accurate as the number of trials increases (according to the law of large numbers).

Applications:

Numerical Integration: When integrals are difficult to evaluate analytically, Monte Carlo methods approximate them by averaging sampled values.
Optimization: Used in scenarios like stochastic optimization, where exact solutions are hard to compute.
Statistical Inference: Monte Carlo is used in Bayesian inference to estimate posterior distributions.
Simulations: For example, in finance to model stock prices or risks.
Basic Process:
Define a Problem: The first step is to define a domain of possible inputs (often high-dimensional and complex).

Generate Random Samples: Randomly generate inputs from a probability distribution over the domain.

Compute Results: Evaluate the function or process for each randomly generated input.

Average the Results: Use the results to compute an average or distribution of outputs, which serves as the approximation to the problem.

Example: Estimating Pi
A common example is using the Monte Carlo method to estimate Pi:

Consider a circle of radius 1 inside a square with sides of length 2.
Generate random points in the square.
The ratio of points that fall inside the circle to the total number of points is approximately Pi/4. By multiplying this ratio by 4, you can estimate the value of Pi.
Advantages:
Scalability: Works well with problems of high-dimensional spaces.
Simplicity: The algorithm is easy to implement and doesn't require detailed knowledge of the problem.
Flexibility: It can be applied to a wide range of problems where deterministic solutions are not feasible.
Disadvantages:
Convergence Time: The algorithm may require a large number of samples to converge to an accurate solution, which can be computationally expensive.
Accuracy: As it is based on random sampling, the result is an approximation and not an exact answer.
Monte Carlo algorithms are especially useful in scenarios where exact mathematical modeling is difficult or impossible, but simulation can provide insights or approximate solutions."""

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
sentences = splitter.split_text(multi_sentence_text)

for i, sentence in enumerate(sentences):
    print(f"#{i+1}: {sentence}")

#1: The Monte Carlo algorithm (or Monte Carlo method) refers to a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. These methods are used in various fields such as finance, physics, engineering, and AI for problems that are deterministic in principle but difficult to solve directly.
#2: Key Aspects of the Monte Carlo Algorithm:
Random Sampling: The core idea is to generate random variables to simulate complex systems or processes. The algorithm uses probabilistic models and generates numerous random samples to approximate the solution to a deterministic problem.
#3: Estimation of Results: Monte Carlo methods provide estimates of the outcome by running many simulations and averaging the results. These results tend to get more accurate as the number of trials increases (according to the law of large numbers).

Applications:
#4: Numerical Integration: When integrals are difficult to evaluate analytically, Monte Carlo methods approx

In [26]:
import numpy as np
import time
from typing import Union
from functools import singledispatchmethod
from copy import copy
import re

from abc import ABC, abstractmethod


def find_end_of_sentence(text, start):
    end = start
    while end < len(text) and text[end] not in '.!?':
        end += 1
    return end


class AbstractSplitter(ABC):
    """ This is how DRY dies, with thunderous applaus of abstract OOP """
    @abstractmethod
    def split(self, data, chunk_size, overlap, margin):
        pass

    @singledispatchmethod
    @abstractmethod
    def _overlap_splits_dispatcher(self, overlap, data_size, chunk_size):
        pass
    
    @singledispatchmethod
    @abstractmethod
    def _split_dispatcher(self, overlap, data, chunk_size, margin):
        pass


class TextSplitter(AbstractSplitter):
    white_space_pattern = re.compile('\s')

    @staticmethod
    def splits(text_length, chunk_size):
        return np.arange(chunk_size, text_length-chunk_size+1, chunk_size)

    @staticmethod
    def _overlap_splits(text_length, chunk_size, overlap):
        return np.arange(chunk_size, text_length-chunk_size+1, chunk_size-overlap)
    
    @singledispatchmethod
    def _split_dispatcher(self, overlap, text, chunk_size, margin):
        raise ValueError(f"Invalid overlap type: {type(overlap)}")
    
    @singledispatchmethod
    def _overlap_splits_dispatcher(self, margin, overlap, data_size, chunk_size):
        raise ValueError(f"Invalid overlap type: {type(margin)}")
    
    @_overlap_splits_dispatcher.register
    def _(self, margin: None, overlap: int, data_size, chunk_size):
        return TextSplitter.overlap_splits(data_size, chunk_size, overlap)

    @_overlap_splits_dispatcher.register
    def _(self, margin: int, overlap: int, data_size, chunk_size):
        overlaps = []

    @_split_dispatcher.register
    def _(self, overlap: int, text, chunk_size, margin):
        max_chunk_size = chunk_size - overlap
        remaining_text = copy(text)
        split_positions = []
        print(f"Text length: {len(remaining_text)}")
        text_length = len(remaining_text)
        static_split_pos = text_length - max_chunk_size
        positive_margin_subset = text[static_split_pos:static_split_pos+margin]
        new_pos = self.split_pos(positive_margin_subset, static_split_pos)
        print("Positive margin subset:", positive_margin_subset)
        print(f"New position {new_pos}")
        split_positions.insert(0, (new_pos, len(remaining_text)))
        remaining_text = remaining_text[:new_pos]
        while True:
            text_length = len(remaining_text)
            static_split_pos = text_length - max_chunk_size
            positive_margin_subset = text[static_split_pos:static_split_pos+margin]
            new_pos = self.split_pos(positive_margin_subset, static_split_pos)
            split_positions.insert(0, (new_pos, len(remaining_text)+overlap))
            print(f"New position {new_pos}")
            remaining_text = remaining_text[:new_pos]
            print(f"Remaining text length: {len(remaining_text)}")
            
            if len(remaining_text) < max_chunk_size:
                break
        
        split_positions.insert(0, (0, len(remaining_text)+overlap))
        print([j-i for i, j in split_positions])
        return {key:text[i:j] for key, (i, j) in enumerate(split_positions)}

    @_split_dispatcher.register
    def _(self, overlap: None, text, chunk_size, margin):
        split_pos = self.splits(len(text), chunk_size)
        split_pos = np.insert(split_pos, 0, 0)
        return {key:text[i:i+chunk_size] for key, i in enumerate(split_pos)}
    
    @_split_dispatcher.register
    def _(self, overlap: float, text, chunk_size, margin):
        return self.overlap_splits(len(text), chunk_size, int(chunk_size*overlap))
    
    def split_pos(self, string, current_position):
        for i, letter in enumerate(string):
            if letter == '.':
                print(string)
                print(i)
                return current_position + i + 1

        for i, letter in enumerate(string):
            if not isinstance(self.white_space_pattern.match(letter), type(None)):
                print(string)
                print(i)
                return current_position + i + 1
            
        return current_position
    
    
    def split_neg(self, string, current_position):
        inv_string = string[::-1]
        for i, letter in enumerate(inv_string):
            if letter == '.':
                print(string)
                print(i)
                return current_position + (len(string)-i)

        for i, letter in enumerate(inv_string):
            if not isinstance(self.white_space_pattern.match(letter), type(None)):
                print(string)
                print(i)
                return current_position + (len(string)-i)
            
        return current_position

    def split(self, text, chunk_size=100, overlap: None = None, margin: int = 10):
        return self._split_dispatcher(overlap, text, chunk_size, margin)
    
    def overlap_splits(self, text_length, chunk_size, overlap, margin=None):
        return self._overlap_splits_dispatcher(margin, overlap, text_length, chunk_size)


ts = TextSplitter()
ts.split(multi_sentence_text, 500, 100, 20)


Text length: 2850
hm may require a lar
2
Positive margin subset: hm may require a lar
New position 2453
By multiplying this 
2
New position 2056
Remaining text length: 2056
 compute an average 
0
New position 1657
Remaining text length: 1657
e, in finance to mod
2
New position 1260
Remaining text length: 1260
bers).

Applications
5
New position 866
Remaining text length: 866
stems or processes. 
18
New position 485
Remaining text length: 485
ional algorithms tha
5
New position 91
Remaining text length: 91
[191, 494, 481, 494, 497, 499, 497, 397]


{0: 'The Monte Carlo algorithm (or Monte Carlo method) refers to a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. These methods are used',
 1: 'algorithms that rely on repeated random sampling to obtain numerical results. These methods are used in various fields such as finance, physics, engineering, and AI for problems that are deterministic in principle but difficult to solve directly.\n\nKey Aspects of the Monte Carlo Algorithm:\nRandom Sampling: The core idea is to generate random variables to simulate complex systems or processes. The algorithm uses probabilistic models and generates numerous random samples to approximate the so',
 2: ' The algorithm uses probabilistic models and generates numerous random samples to approximate the solution to a deterministic problem.\n\nEstimation of Results: Monte Carlo methods provide estimates of the outcome by running many simulations and averaging the results. These results tend to 

In [21]:
from time import time_ns
import re

white_space_pattern = re.compile('\s')
dot_pattern = re.compile('.')

def simple_subset(string, start, end):
    for i, letter in enumerate(string[start:end]):
        if letter == '.':
            return start + i

    for i, letter in enumerate(string[start:end]):
        if white_space_pattern.match(letter):
            return start + i
        
    return start



def simple_subset_alt(string, start, end):
    for i, letter in enumerate(string[start:end]):
        if letter == '.':
            return start + i

    for i, letter in enumerate(string[start:end]):
        if not isinstance(white_space_pattern.match(letter), type(None)):
            return start + i
        
    return start

string = "The Monte Carlo algorithm (or Monte Carlo method) refers to a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results."

start_time = time_ns()
[simple_subset(multi_sentence_text, 4, 500) for _ in range(0, 100000)]
print(f"Time needed for execution: {time_ns()-start_time}")

start_time = time_ns()
[simple_subset_alt(multi_sentence_text, 4, 500) for _ in range(0, 100000)]
print(f"Time needed for execution: {time_ns()-start_time}")



Time needed for execution: 329113500
Time needed for execution: 325284100
